In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import ast

In [2]:
# data = pd.read_csv('all_games.csv')
# data['Possession'] = pd.to_numeric(data['Possession'], errors='coerce')  # Convert to float
# data['AwayScore'] = pd.to_numeric(data['AwayScore'], errors='coerce')
# data['HomeScore'] = pd.to_numeric(data['HomeScore'], errors='coerce')
# games_to_remove = data.groupby('GameID')['Possession'].apply(lambda x: x.isna().all())
# game_ids_to_remove = games_to_remove[games_to_remove].index
# data = data[~data['GameID'].isin(game_ids_to_remove)]
# data['ActivePlayers'] = data['ActivePlayers'].apply(ast.literal_eval)

In [3]:
# shot_type_dict = {'3-pt jump shot': 0, '3-pt shot': 0, '3-pt layup': 0, '3-pt dunk': 0, '3-pt hook shot': 0, '3-pt tip-in': 0, '2-pt shot': 1, '2-pt jump shot': 1, '2-pt hook shot': 1, '2-pt tip-in': 2, '2-pt dunk': 2, '2-pt layup': 2, 'no shot': 3}
# shot_dist_dict = {}
# for i in range(95):
#     if i > 39:
#         shot_dist_dict[i] = 8
#     elif i > 27:
#         shot_dist_dict[i] = 7
#     elif i > 22:
#         shot_dist_dict[i] = 6
#     elif i > 21:
#         shot_dist_dict[i] = 5
#     elif i > 16:
#         shot_dist_dict[i] = 4
#     elif i > 10:
#         shot_dist_dict[i] = 3
#     elif i > 3:
#         shot_dist_dict[i] = 2
#     else:
#         shot_dist_dict[i] = 1

# all_tokens = []
# shot_types = []
# for i in range(len(data)):
#     if i % 1000000 == 0:
#         print(i)
#     shot_type = None
#     tokenized = []
#     if not pd.isna(data['AwayEvent'].iloc[i]):
#         if 'makes' in data['AwayEvent'].iloc[i] or 'misses' in data['AwayEvent'].iloc[i]:
#             tokenized.append(0) # play type
#             if 'makes' in data['AwayEvent'].iloc[i]:
#                 tokenized.append(1) # make
#             else:
#                 tokenized.append(0) # miss
#             tokenized.append(player_dict[data['ActivePlayers'].iloc[i][0]]) # player id
#             if 'free throw' in data['AwayEvent'].iloc[i]:
#                 tokenized.append(0) # shot type
#                 tokenized.append(2) # shot distance (11-16)
#                 tokenized.append(0) # assist
#                 tokenized.append(0) # block
#             else:
#                 if 'makes' in data['AwayEvent'].iloc[i]:
#                     shot_type = re.search(r"makes (.*?)(?: (?:from|at)|$)", data['AwayEvent'].iloc[i]).group(1).strip()
#                 else:
#                     shot_type = re.search(r"misses (.*?)(?: (?:from|at)|$)", data['AwayEvent'].iloc[i]).group(1).strip()
#                 tokenized.append(shot_type_dict[shot_type]) # shot type
#                 if 'at rim' in data['AwayEvent'].iloc[i]:
#                     tokenized.append(1) # shot distance (0-3)
#                 else:
#                     shot_dist = re.search(r"from\s+(\w+)", data['AwayEvent'].iloc[i])
#                     if shot_dist:
#                         tokenized.append(shot_dist_dict[int(shot_dist.group(1))]) # shot distance
#                     else:
#                         tokenized.append(0) # no shot distance
#                 if 'assist' in data['AwayEvent'].iloc[i]:
#                     if 'block' in data['AwayEvent'].iloc[i]:
#                         print('error')
#                     tokenized.append(player_dict[data['ActivePlayers'].iloc[i][1]]) # assist
#                     tokenized.append(0) # block
#                 elif 'block' in data['AwayEvent'].iloc[i]:
#                     tokenized.append(0) # assist
#                     tokenized.append(player_dict[data['ActivePlayers'].iloc[i][1]]) # block
#     elif not pd.isna(data['HomeEvent'].iloc[i]) and len(data['ActivePlayers'].iloc[i]) > 0:
#         if 'makes' in data['HomeEvent'].iloc[i] or 'misses' in data['HomeEvent'].iloc[i]:
#             tokenized.append(0) # play type
#             if 'makes' in data['HomeEvent'].iloc[i]:
#                 tokenized.append(1) # make
#             else:
#                 tokenized.append(0) # miss
#             tokenized.append(player_dict[data['ActivePlayers'].iloc[i][0]]) # player id
#             if 'free throw' in data['HomeEvent'].iloc[i]:
#                 tokenized.append(0) # shot type
#                 tokenized.append(2) # shot distance (11-16)
#                 tokenized.append(0) # assist
#                 tokenized.append(0) # block
#             else:
#                 if 'makes' in data['HomeEvent'].iloc[i]:
#                     shot_type = re.search(r"makes (.*?)(?: (?:from|at)|$)", data['HomeEvent'].iloc[i]).group(1).strip()
#                 else:
#                     shot_type = re.search(r"misses (.*?)(?: (?:from|at)|$)", data['HomeEvent'].iloc[i]).group(1).strip()
#                 tokenized.append(shot_type_dict[shot_type]) # shot type
#                 if 'at rim' in data['HomeEvent'].iloc[i]:
#                     tokenized.append(1) # shot distance (0-3)
#                 else:
#                     shot_dist = re.search(r"from\s+(\w+)", data['HomeEvent'].iloc[i])
#                     if shot_dist:
#                         tokenized.append(shot_dist_dict[int(shot_dist.group(1))]) # shot distance
#                     else:
#                         tokenized.append(0) # no shot distance
#                 if 'assist' in data['HomeEvent'].iloc[i]:
#                     if 'block' in data['HomeEvent'].iloc[i]:
#                         print('error')
#                     tokenized.append(player_dict[data['ActivePlayers'].iloc[i][1]]) # assist
#                     tokenized.append(0) # block
#                 elif 'block' in data['HomeEvent'].iloc[i]:
#                     tokenized.append(0) # assist
#                     tokenized.append(player_dict[data['ActivePlayers'].iloc[i][1]]) # block
#     data.at[i, 'shot_type'] = shot_type
#     data.at[i, 'token'] = tokenized
# #     shot_types.append(shot_type)
# #     all_tokens.append(tokenized)
# # data['shot_type'] = shot_types
# # data['token'] = all_tokens

In [4]:
# for i in range(len(data)):
#     tokenized = []
#     if (not pd.isna(data['AwayEvent'].iloc[i]) and 'Defensive rebound' in data['AwayEvent'].iloc[i]) or (not pd.isna(data['HomeEvent'].iloc[i]) and 'Defensive rebound' in data['HomeEvent'].iloc[i]):
#         tokenized.append(1) # play type
#         tokenized.append(1) # offensive or defensive
#         if len(data['ActivePlayers'].iloc[i]) > 0:
#             tokenized.append(player_dict[data['ActivePlayers'].iloc[i][0]]) # player ID
#         else:
#             tokenized.append(0)
#         data.at[i, 'token'] = tokenized
#     elif (not pd.isna(data['AwayEvent'].iloc[i]) and 'Offensive rebound' in data['AwayEvent'].iloc[i]) or (not pd.isna(data['HomeEvent'].iloc[i]) and 'Offensive rebound' in data['HomeEvent'].iloc[i]):
#         tokenized.append(1) # play type
#         tokenized.append(0) # offensive or defensive
#         if len(data['ActivePlayers'].iloc[i]) > 0:
#             tokenized.append(player_dict[data['ActivePlayers'].iloc[i][0]]) # player ID
#         else:
#             tokenized.append(0)
#         data.at[i, 'token'] = tokenized

In [5]:
# turnover_dict = {'isolation violation.': 0, 'lost ball': 1, 'bad pass': 1, '3 second': 2, 'discontinued dribble': 1, 'no': 0, 'foul': 3, 'off goaltending': 5, 'offensive foul': 3, 'traveling': 1, 'out of bounds': 4, 'shot clock': 2, 'back court': 4, 'palming': 1, 'step out of bounds': 4, 'poss. lost ball': 1, 'out of bounds lost ball': 4, '3 sec': 2, 'kicked ball': 0, 'offensive goaltending': 5, 'inbound': 6, 'turnover': 0, '8 sec': 2, 'dbl dribble': 1, '5 sec': 2, 'lane violation.': 2, 'illegal screen': 3, 'illegal assist': 0, 'jump ball violation.': 6, '5 sec inbounds': 2, '10 second': 2, 'illegal pick': 3, 'lane violation': 2, 'double dribble': 1, 'isolation violation': 2, 'double personal': 3, 'punched ball': 0, 'post up': 6, 'stolen pass': 1, 'enter basket from below': 4, 'score in opp. basket': 5, 'swinging elbows': 0, 'jump ball violation': 6, 'excessive timeout': 6}

# for i in range(10000000, len(data)):
#     if i % 1000000 == 0:
#         print(i)
#     tokenized = []
#     if not pd.isna(data['HomeEvent'].iloc[i]) and 'Turnover' in data['HomeEvent'].iloc[i]:
#         tokenized.append(2) # play type
#         if len(data['ActivePlayers'].iloc[i]) == 0:
#             tokenized.append(0)
#         else:
#             tokenized.append(player_dict[data['ActivePlayers'].iloc[i][0]]) # player ID
#         if 'steal' in data['HomeEvent'].iloc[i]:
#             tokenized.append(player_dict[data['ActivePlayers'].iloc[i][1]]) # steal
#             tokenized.append(turnover_dict[re.search(r"\((.*?);", data['HomeEvent'].iloc[i]).group(1)]) # turnover reason
#         else:
#             tokenized.append(0) # no steal
#             tokenized.append(turnover_dict[re.search(r"\((.*?)\)", data['HomeEvent'].iloc[i]).group(1)]) # turnover reason
#         data.at[i, 'token'] = tokenized
#     elif not pd.isna(data['AwayEvent'].iloc[i]) and 'Turnover' in data['AwayEvent'].iloc[i]:
#         tokenized.append(2) # play type
#         if len(data['ActivePlayers'].iloc[i]) == 0:
#             tokenized.append(0)
#         else:
#             tokenized.append(player_dict[data['ActivePlayers'].iloc[i][0]]) # player ID
#         if 'steal' in data['AwayEvent'].iloc[i]:
#             tokenized.append(player_dict[data['ActivePlayers'].iloc[i][1]]) # steal
#             tokenized.append(turnover_dict[re.search(r"\((.*?);", data['AwayEvent'].iloc[i]).group(1)]) # turnover reason
#         else:
#             tokenized.append(0) # no steal
#             tokenized.append(turnover_dict[re.search(r"\((.*?)\)", data['AwayEvent'].iloc[i]).group(1)]) # turnover reason
#         data.at[i, 'token'] = tokenized

In [6]:
# for i in range(1, len(data)):
#     if not pd.isna(data['AwayEvent'].iloc[i]) and 'Start of 1st' in data['AwayEvent'].iloc[i]:
#         data.at[i, 'token'] = [3]

In [7]:
# for i in range(len(data)):
#     if i % 1000000 == 0:
#         print(i)
#     if len(data['token'].iloc[i]) > 0 and len(data['token'].iloc[i]) < 7:
#         data.at[i, 'token'] = np.pad(data['token'].iloc[i], (0, 7 - len(data['token'].iloc[i])), mode='constant', constant_values=0)

In [8]:
# data.to_feather('all_games.feather')

In [9]:
data = pd.read_feather('all_games.feather')
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

In [10]:
data = data.sort_values(by=['Date', 'GameID', 'Possession'])
data = data.reset_index(drop=True)

In [11]:
all_players = pd.concat([data['A1'], data['A2'], data['A3'], data['A4'], data['A5'], data['H1'], data['H2'], data['H3'], data['H4'], data['H5']]).dropna()
all_players = all_players.unique()
player_dict = {player_id: idx + 1 for idx, player_id in enumerate(all_players)}

In [ ]:
data['token'] = data['token'].apply(lambda x: [] if len(x) > 0 and x[0] == 3 else x)
new_rows = []
flag = 0

# Iterate over the rows of the dataframe
for i in range(len(data)):
    if i % 1000000 == 0:
        print(i)

    if i > 0 and data.iloc[i]['Period'] != data.iloc[i - 1]['Period']:
        new_row = {'Period': None}

    # Check if a new row needs to be added
    if i > 0 and data.iloc[i]['Period'] == 1 and data.iloc[i - 1]['Period'] >= 4:
        # Create the new row
        flag = 0
        new_row = {'Period': None, 'token': [3, 0, 0, 0, 0, 0, 0]}
        new_rows.append(new_row)
    elif i > 0 and data.iloc[i]['Period'] == 2 and data.iloc[i - 1]['Period'] >= 4:
        flag = 1
        new_row = {'Period': None, 'token': [3, 0, 0, 0, 0, 0, 0]}
        new_rows.append(new_row)

    # Add the current row
    if flag == 1:
        data.at[i, 'Period'] -= 1

    new_rows.append(data.iloc[i].to_dict())

data = pd.DataFrame(new_rows).reset_index(drop=True)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000


In [52]:
token_set = data[data['token'].apply(len) > 0]
tensor = np.stack(token_set['token'].values)
new_tensor = np.empty((tensor.shape[0], 5))
for i, r in enumerate(tensor):
    if r[0] == 0:
        new_tensor[i] = [r[3], r[2], r[4]-1, r[1], r[5] + r[6]]
    elif r[0] == 1:
        new_tensor[i] = [4 + r[1], r[2], -1, -1, -1]
    elif r[0] == 2:
        new_tensor[i] = [6 + r[3], r[1], -1, -1, r[2]]
    elif r[0] == 3:
        new_tensor[i] = [-1, -1, -1, -1, -1]

np.save('tensor.npy', new_tensor + 1)

In [41]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', None)

In [53]:
data['Period'].value_counts()

Period
4.0    3229586
2.0    3185630
3.0    3005012
1.0    2953483
5.0      77378
6.0       8530
7.0       1387
8.0        251
Name: count, dtype: int64

In [47]:
data.iloc[599893:599900]

,PlayNum,GameID,Date,Period,Possession,Time,AwayName,AwayScore,AwayEvent,HomeName,HomeScore,HomeEvent,AwayIn,AwayOut,HomeIn,HomeOut,ActivePlayers,A1,A2,A3,A4,A5,H1,H2,H3,H4,H5,shot_type,token
599893,420.0,200111100MIA,2001-11-10,4.0,193.0,0:00.0,SAC,90.0,None,MIA,75.0,Offensive rebound by Team,None,None,None,None,[],divacvl01,chrisdo01,jacksbo01,stojape01,bibbymi01,hamilta01,macksa01,houseed01,stepavl01,gatlich01,None,"[1, 0, 0, 0, 0, 0, 0]"
599894,421.0,200111100MIA,2001-11-10,4.0,193.0,0:00.0,SAC,90.0,End of 4th quarter,MIA,75.0,End of 4th quarter,None,None,None,None,[],divacvl01,chrisdo01,jacksbo01,stojape01,bibbymi01,hamilta01,macksa01,houseed01,stepavl01,gatlich01,None,[]
599895,0.0,200111100MIL,2001-11-10,0.0,0.0,12:00.0,MIN,0.0,Start of 1st quarter,MIL,0.0,Start of 1st quarter,None,None,None,None,[],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,None,[]
599896,1.0,200111100MIL,2001-11-10,0.0,0.0,12:00.0,MIN,0.0,Jump ball: M. Pope vs. K. Garnett (T. Brandon gains possession),MIL,0.0,Jump ball: M. Pope vs. K. Garnett (T. Brandon gains possession),None,None,None,None,[],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,None,[]
599897,2.0,200111100MIL,2001-11-10,0.0,1.0,11:42.0,MIN,0.0,J. Smith misses 2-pt jump shot from 16 ft,MIL,0.0,None,None,None,None,None,[smithjo02],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,2-pt jump shot,"[0, 0, 347, 1, 3, 0, 0]"
599898,3.0,200111100MIL,2001-11-10,0.0,1.0,11:40.0,MIN,0.0,Offensive rebound by R. Nesterović,MIL,0.0,None,None,None,None,None,[nestera01],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,None,"[1, 0, 357, 0, 0, 0, 0]"
599899,4.0,200111100MIL,2001-11-10,0.0,1.0,11:29.0,MIN,0.0,T. Brandon misses 2-pt jump shot from 11 ft,MIL,0.0,None,None,None,None,None,[brandte01],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,2-pt jump shot,"[0, 0, 281, 1, 3, 0, 0]"


In [45]:
token_set.iloc[456340:456390]

,PlayNum,GameID,Date,Period,Possession,Time,AwayName,AwayScore,AwayEvent,HomeName,HomeScore,HomeEvent,AwayIn,AwayOut,HomeIn,HomeOut,ActivePlayers,A1,A2,A3,A4,A5,H1,H2,H3,H4,H5,shot_type,token
599892,419.0,200111100MIA,2001-11-10,4.0,193.0,0:00.1,SAC,90.0,None,MIA,75.0,S. Mack misses 3-pt jump shot from 25 ft,None,None,None,None,[macksa01],divacvl01,chrisdo01,jacksbo01,stojape01,bibbymi01,hamilta01,macksa01,houseed01,stepavl01,gatlich01,3-pt jump shot,"[0, 0, 1845, 0, 6, 0, 0]"
599893,420.0,200111100MIA,2001-11-10,4.0,193.0,0:00.0,SAC,90.0,None,MIA,75.0,Offensive rebound by Team,None,None,None,None,[],divacvl01,chrisdo01,jacksbo01,stojape01,bibbymi01,hamilta01,macksa01,houseed01,stepavl01,gatlich01,None,"[1, 0, 0, 0, 0, 0, 0]"
599897,2.0,200111100MIL,2001-11-10,0.0,1.0,11:42.0,MIN,0.0,J. Smith misses 2-pt jump shot from 16 ft,MIL,0.0,None,None,None,None,None,[smithjo02],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,2-pt jump shot,"[0, 0, 347, 1, 3, 0, 0]"
599898,3.0,200111100MIL,2001-11-10,0.0,1.0,11:40.0,MIN,0.0,Offensive rebound by R. Nesterović,MIL,0.0,None,None,None,None,None,[nestera01],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,None,"[1, 0, 357, 0, 0, 0, 0]"
599899,4.0,200111100MIL,2001-11-10,0.0,1.0,11:29.0,MIN,0.0,T. Brandon misses 2-pt jump shot from 11 ft,MIL,0.0,None,None,None,None,None,[brandte01],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,2-pt jump shot,"[0, 0, 281, 1, 3, 0, 0]"
599900,5.0,200111100MIL,2001-11-10,0.0,2.0,11:27.0,MIN,0.0,None,MIL,0.0,Defensive rebound by S. Cassell,None,None,None,None,[cassesa01],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,None,"[1, 1, 1400, 0, 0, 0, 0]"
599901,6.0,200111100MIL,2001-11-10,0.0,2.0,11:18.0,MIN,0.0,None,MIL,0.0,G. Robinson misses 3-pt jump shot from 26 ft,None,None,None,None,[robingl01],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,3-pt jump shot,"[0, 0, 1176, 0, 6, 0, 0]"
599902,7.0,200111100MIL,2001-11-10,0.0,3.0,11:17.0,MIN,0.0,Defensive rebound by T. Brandon,MIL,0.0,None,None,None,None,None,[brandte01],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,None,"[1, 1, 281, 0, 0, 0, 0]"
599903,8.0,200111100MIL,2001-11-10,0.0,3.0,11:09.0,MIN,0.0,R. Nesterović misses 2-pt hook shot from 4 ft,MIL,0.0,None,None,None,None,None,[nestera01],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,2-pt hook shot,"[0, 0, 357, 1, 2, 0, 0]"
599904,9.0,200111100MIL,2001-11-10,0.0,4.0,11:07.0,MIN,0.0,None,MIL,0.0,Defensive rebound by G. Robinson,None,None,None,None,[robingl01],brandte01,nestera01,szczewa02,smithjo02,garneke01,allenra02,robingl01,popema01,cassesa01,masonan01,None,"[1, 1, 1176, 0, 0, 0, 0]"


In [49]:
# Filter rows where 'Possession' is 1 and count occurrences of each 'Period' value
period_counts = data.loc[data['Possession'] == 1, 'Period'].value_counts().to_dict()

# Output the result
print(period_counts)

{1.0: 37959, 2.0: 27}
